In [ ]:
import numpy as np
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Διαβάζουμε τα αρχεία του διαγωνισμού
Εδώ δεν χρειάζεται να πειράξετε κάτι

In [ ]:
sample_submission_file = "/kaggle/input/eeestech-challenge-task-1/sample_submission.csv"
train_file = "/kaggle/input/eeestech-challenge-task-1/train.csv"
test_file = "/kaggle/input/eeestech-challenge-task-1/test.csv"

In [ ]:
train_data = pd.read_csv(train_file, header=None) 
test_data = pd.read_csv(test_file, header=None) 
train_data.head(100)

In [ ]:
test_data.head()

# Δημιουργούμε πινακες και dataframes
Εδώ δεν χρειάζεται να πειράξετε κάτι

In [ ]:
train_labels = train_data.iloc[:, [1]].values
train_features = train_data.iloc[:, 2:].values
test_names = test_data.iloc[:, [0]]
test_features = test_data.iloc[:, 1:].values

# Ορίζουμε και εκπαιδεύουμε έναν ταξιμονητή
Αυτό είναι το κομμάτι που θα αλλάζετε δοκιμάζοντας διάφορα μοντέλα και παραμέτρους.
Επειδή το test.csv δεν έχει ετικέτες, δεν μπορεί να χρησιμοποιηθεί για test κατά την εκπαίδευση για να εκτιμήσουμε τη γενίκευση του μοντέλου.
Θα πρέπει να χρησιμοποιήσουμε ένα κομμάτι του train.csv για train και ένα για test.

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train_features, train_labels, test_size=0.33)

Εκπαιδεύουμε έναν ταξινομητή στο X_train, y_train

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

sc = StandardScaler()

train_scaled = sc.fit_transform(X_train)
test_scaled = sc.fit_transform(X_test)

logisticRegr = LogisticRegression(max_iter = 2000)
logisticRegr.fit(X_train, y_train.ravel())

preds = logisticRegr.predict(X_test)

Κάνουμε προβλέψεις στο X_test, y_test και παίρνουμε μια εκτίμηση της ικανότητας γενίκευσης του ταξινομητή.

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, preds))

## Δοκιμάζουμε με grid search

In [ ]:
from sklearn.feature_selection import VarianceThreshold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import Normalizer, StandardScaler, MinMaxScaler, PowerTransformer, MaxAbsScaler, LabelEncoder
from sklearn.model_selection import GridSearchCV

In [ ]:

#choosing parameters for the grid
parameters = {'scaler': [StandardScaler(), MinMaxScaler(), MaxAbsScaler()],
    'selector__threshold': [0, 0.001, 0.01],
    'classifier__solver': ['lbfgs', 'liblinear', 'saga'],
    'classifier__C': [100, 10, 1.0, 0.1],
    'classifier__max_iter': [2000, 2500, 3000]
}

#creating pipeline
pipe = Pipeline([
  ('scaler', StandardScaler()),
  ('selector', VarianceThreshold()),
  ('classifier', LogisticRegression())
])

grid = GridSearchCV(pipe, 
                    parameters, 
                    cv = 10,
                    ).fit(X_train, y_train.ravel())

In [ ]:
best_params = grid.best_params_

print("Optimal Parameters:\n")

for k in best_params:
  print(k.title().replace("__", " "), ":", best_params[k])

In [ ]:
parameters = {'scaler': [StandardScaler()],
	'selector__threshold': [0],	
  'classifier__solver': ['lbfgs'],
	'classifier__C': [100],
  'classifier__max_iter': [2000]
}

start_time = time.time()
grid = GridSearchCV(pipe, 
                    parameters, 
                    cv = 10,
                    ).fit(train, train_labels)
train_time['logistic regression'] = np.round(time.time() - start_time, 4)

In [ ]:
start_time = time.time()
logistic_preds = grid.predict(test)
test_time['logistic regression'] = np.round(time.time() - start_time,4)

optimized_acc['logistic regression'] = logisticRegr.score(test, test_labels)
optimized_f1['logistic regression'] = f1_score(test_labels, logistic_preds)

print("\nClassification Accuracy: \n")
sorted_acc = [(k, optimized_acc[k]) for k in sorted(optimized_acc, key = optimized_acc.get, reverse = True)]

for k, v in sorted_acc:
  print(k.title(), ":", np.round(100 * v, 3), "%")

print("\nClassification F1: \n")
sorted_f1 = [(k, optimized_f1[k]) for k in sorted(optimized_f1, key = optimized_f1.get, reverse = True)]

for k, v in sorted_f1:
  print(k.title(), ":", np.round(100 * v, 3), "%")

print("__________________________________________________")

print("\nTimes: ")

print("\nTrain times:\n")

for k,v in train_time.items():
  print(k.title(), "train time is", v)
print("\nTest times:\n")
for k,v in test_time.items():
  print(k.title(), "test time is", v)

## Δοκιμάζουμε KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score

In [ ]:
tmp = list(range(1, 100))
neighbors = list(filter(lambda x: x % 2 != 0, tmp))

#choosing parameters for the grid
parameters = {'scaler': [StandardScaler(), MinMaxScaler(), MaxAbsScaler()],
    'selector__threshold': [0, 0.001, 0.01],
    'classifier__n_neighbors': neighbors,
}

#creating pipeline
pipe = Pipeline([
  ('scaler', StandardScaler()),
  ('selector', VarianceThreshold()),
  ('classifier', KNeighborsClassifier())
])

grid = GridSearchCV(pipe, 
                    parameters, 
                    cv = 10,
                    ).fit(X_train, y_train.ravel())

In [ ]:
best_params = grid.best_params_

print("Optimal Parameters:\n")

for k in best_params:
  print(k.title().replace("__", " "), ":", best_params[k])

In [ ]:
parameters = {'scaler': [MinMaxScaler()],
              'selector__threshold': [0.001],
              'classifier__n_neighbors': [1],
}

grid = GridSearchCV(pipe, 
                    parameters, 
                    cv = 10,
                    ).fit(X_train, y_train.ravel())

In [ ]:
from sklearn.metrics import classification_report

knn_preds = grid.predict(X_test)
print(classification_report(y_test.ravel(), knn_preds))

## Overasampling:

In [ ]:
from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler()
ov_X_train, ov_y_train = ros.fit_resample(X_train, y_train)

In [ ]:
neighbors = list(range(1, 50))

#choosing parameters for the grid
parameters = {'scaler': [StandardScaler(), MinMaxScaler(), MaxAbsScaler()],
    'selector__threshold': [0, 0.001, 0.01],
    'classifier__n_neighbors': neighbors,
}

#creating pipeline
pipe = Pipeline([
  ('scaler', StandardScaler()),
  ('selector', VarianceThreshold()),
  ('classifier', KNeighborsClassifier())
])

grid = GridSearchCV(pipe, 
                    parameters, 
                    cv = 10,
                    ).fit(ov_X_train, ov_y_train.ravel())

In [ ]:
best_params = grid.best_params_

print("Optimal Parameters:\n")

for k in best_params:
  print(k.title().replace("__", " "), ":", best_params[k])

In [ ]:
parameters = {'scaler': [MaxAbsScaler()],
              'selector__threshold': [0.01],
              'classifier__n_neighbors': [1],
}

grid = GridSearchCV(pipe, 
                    parameters, 
                    cv = 10,
                    ).fit(ov_X_train, ov_y_train.ravel())

In [ ]:
ov_knn_preds = grid.predict(X_test)
print(classification_report(y_test.ravel(), ov_knn_preds))

## Gradient Booster

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report
from sklearn.model_selection import KFold
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
gradient_booster = GradientBoostingClassifier(learning_rate=0.1)
gradient_booster.get_params()

In [ ]:
gradient_booster.fit(ov_X_train,ov_y_train)
ov_gradient_preds = gradient_booster.predict(X_test)
print(classification_report(y_test.ravel(),ov_gradient_preds))

## GRADIENT BOOSTER ΜE GRID SEARCH

In [ ]:
estimators = list(range(20,81,10))
#choosing parameters for the grid
parameters = {'scaler': [StandardScaler(), MinMaxScaler(), MaxAbsScaler()],
    'selector__threshold': [0, 0.001, 0.01],
    'classifier__n_estimators': estimators,
    'classifier__learning_rate': [0.01, 0.1, 0.9]
}

#creating pipeline
pipe = Pipeline([
  ('scaler', StandardScaler()),
  ('selector', VarianceThreshold()),
  ('classifier', GradientBoostingClassifier())
])

grid = GridSearchCV(pipe, 
                    parameters, 
                    cv = 10,
                    ).fit(ov_X_train, ov_y_train.ravel())

In [ ]:
parameters = {'scaler': [MaxAbsScaler()],
              'selector__threshold': [0.01],
              'classifier__n_neighbors': [1],
}

grid = GridSearchCV(pipe, 
                    parameters, 
                    cv = 10,
                    ).fit(ov_X_train, ov_y_train.ravel())

# Φτιάχνουμε κατάλληλα το αρχείο submission.csv με τα predictions μας
Εδώ δεν χρειάζεται να πειράξετε κάτι

In [ ]:
predictions_df = pd.DataFrame(final_predictions, columns = ['col2'])
test_names.columns=['col1']
submission_df = pd.concat([test_names, predictions_df], axis=1)
header_columns = {'col1': ["ID"], 'col2': ["EXPECTED"]}
header = pd.DataFrame(data=header_columns)
submission_df = pd.concat([header, submission_df])
submission_df.to_csv('submission.csv', index=False, header=False)